In [1]:
import pandapower as pp
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Create a simple power network using pandapower
net = pp.networks.example_simple()

# Define a neural network model
class PowerFlowNN(nn.Module):
    def __init__(self):
        super(PowerFlowNN, self).__init__()
        # self.fc1 = nn.Linear(8, 10)  # Adjust input size to match the concatenated input size
        # self.fc2 = nn.Linear(10, 10)
        # self.fc3 = nn.Linear(10, 8)  # Output size should match the number of buses * 2 (voltage magnitudes and angles)

        self.fc1 = nn.Linear(2, 1000)  # Adjust input size to match the concatenated input size
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 14)  # Output size should match the number of buses

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Function to perform power flow calculation using Newton-Raphson method
def power_flow_calculation(net):
    pp.runpp(net, algorithm='nr')
    return np.concatenate((net.res_bus.vm_pu.values, net.res_bus.va_degree.values))

# Generate training data
def generate_training_data(net, num_samples=1000):
    inputs = []
    targets = []
    for _ in range(num_samples):
        # Randomly perturb the load values
        net.load.p_mw += (np.random.rand(len(net.load)) - 0.5) * 0.1
        net.load.q_mvar += (np.random.rand(len(net.load)) - 0.5) * 0.1
        
        # Perform power flow calculation
        target = power_flow_calculation(net)
        
        # Store the input and target values
        input_values = np.concatenate((net.load.p_mw.values, net.load.q_mvar.values))
        inputs.append(input_values)
        targets.append(target)
    
    inputs = torch.tensor(inputs, dtype=torch.float32)
    targets = torch.tensor(targets, dtype=torch.float32)
    
    return inputs, targets

# Initialize the neural network model
model = PowerFlowNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate training data
inputs, targets = generate_training_data(net)

# Print shapes of inputs and targets to verify
print("Inputs shape: ", inputs.shape)
print("Targets shape: ", targets.shape)

# Train the neural network model
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training completed.")

# Compare the predicted values of the neural network model with the real network
def compare_predictions_with_real_network(net, model):
    # Perform power flow calculation on the real network
    real_values = power_flow_calculation(net)
    
    # Prepare input data for the neural network model
    input_values = np.concatenate((net.load.p_mw.values, net.load.q_mvar.values))
    input_tensor = torch.tensor(input_values, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    # Get predictions from the neural network model
    model.eval()
    with torch.no_grad():
        predicted_values = model(input_tensor).squeeze().numpy()
    
    # Split predicted values into voltage magnitudes and angles
    predicted_vm_pu = predicted_values[:len(net.res_bus)]
    predicted_va_degree = predicted_values[len(net.res_bus):]
    
    # Print predicted voltage angles for comparison
    print("Predicted Voltage Angles: ", predicted_va_degree)

    print("Reference voltage angles:", real_values[7:])

    print("Predicted Voltage Magnitudes: ", predicted_vm_pu)

    print("Reference voltage magnitudes: ", real_values[0:7])

    

# Compare predictions with real network values
compare_predictions_with_real_network(net, model)


C:\Users\bmamudi\AppData\Local\Temp\ipykernel_8540\3849427457.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  inputs = torch.tensor(inputs, dtype=torch.float32)


Inputs shape:  torch.Size([1000, 2])
Targets shape:  torch.Size([1000, 14])
Epoch [10/100], Loss: 1082.0227
Epoch [20/100], Loss: 283.7551
Epoch [30/100], Loss: 61.0294
Epoch [40/100], Loss: 33.1613
Epoch [50/100], Loss: 28.4474
Epoch [60/100], Loss: 11.0173
Epoch [70/100], Loss: 5.6392
Epoch [80/100], Loss: 4.3672
Epoch [90/100], Loss: 3.6069
Epoch [100/100], Loss: 3.4664
Training completed.
Predicted Voltage Angles:  [  52.2654     52.21949    52.037624 -103.28189  -103.14893  -103.093
 -103.12826 ]
Reference voltage angles: [ 50.          50.02293765  50.02293765 -98.57401537 -98.57401537
 -98.49650112 -98.49274608]
Predicted Voltage Magnitudes:  [1.0753058 1.0470281 1.0043347 1.0510114 1.0712448 0.931118  1.1206894]
Reference voltage magnitudes:  [1.02       1.02077734 1.02077734 1.02472549 1.02472549 1.03
 1.02256881]


In [2]:
pp.runpp(net, algorithm='nr')

# Prepare input
Ybus = net._ppc["internal"]["Ybus"].toarray()
S = net._ppc["internal"]["Sbus"]
input_tensor = torch.FloatTensor(np.concatenate([
    Ybus.real.flatten(), 
    Ybus.imag.flatten(),
    S.real, 
    S.imag
]))

V_mag_ref = net.res_bus.vm_pu.values 
V_ang_ref = net.res_bus.va_degree.values

print("Reference voltage magnitudes:", V_mag_ref)
print("Reference voltage angles:", V_ang_ref)

Reference voltage magnitudes: [1.02       1.02077734 1.02077734 1.02472549 1.02472549 1.03
 1.02256881]
Reference voltage angles: [ 50.          50.02293765  50.02293765 -98.57401537 -98.57401537
 -98.49650112 -98.49274608]
